In [18]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('cannabis.csv')

In [5]:
df

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."
...,...,...,...,...,...,...
2346,Zeus-Og,hybrid,4.7,"Happy,Uplifted,Relaxed,Euphoric,Energetic","Earthy,Woody,Pine",Zeus OG is a hybrid cross between Pineapple OG...
2347,Zkittlez,indica,4.6,"Relaxed,Happy,Euphoric,Uplifted,Sleepy","Sweet,Berry,Grape",Zkittlez is an indica-dominant mix of Grape Ap...
2348,Zombie-Kush,indica,5.0,"Relaxed,Sleepy,Talkative,Euphoric,Happy","Earthy,Sweet,Spicy/Herbal",Zombie Kush by Ripper Seeds comes from two dif...
2349,Zombie-Og,indica,4.4,"Relaxed,Sleepy,Euphoric,Happy,Hungry","Sweet,Earthy,Pungent",If you’re looking to transform into a flesh-ea...


What's the Goal?
- Target -> Strain
- X: Effects & Flavor
- Description -> Score?

In [6]:
y = df['Strain']
x = df.drop(columns=['Strain']).copy()
print(y.shape)
print(x.shape)

(2351,)
(2351, 5)


In [7]:
y.isnull().sum()

0

In [8]:
x.isnull().sum()

Type            0
Rating          0
Effects         0
Flavor         46
Description    33
dtype: int64

In [9]:
x = x.dropna()
x

,Type,Rating,Effects,Flavor,Description
0,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."
...,...,...,...,...,...
2346,hybrid,4.7,"Happy,Uplifted,Relaxed,Euphoric,Energetic","Earthy,Woody,Pine",Zeus OG is a hybrid cross between Pineapple OG...
2347,indica,4.6,"Relaxed,Happy,Euphoric,Uplifted,Sleepy","Sweet,Berry,Grape",Zkittlez is an indica-dominant mix of Grape Ap...
2348,indica,5.0,"Relaxed,Sleepy,Talkative,Euphoric,Happy","Earthy,Sweet,Spicy/Herbal",Zombie Kush by Ripper Seeds comes from two dif...
2349,indica,4.4,"Relaxed,Sleepy,Euphoric,Happy,Hungry","Sweet,Earthy,Pungent",If you’re looking to transform into a flesh-ea...


In [11]:
x.isnull().sum()

Type           0
Rating         0
Effects        0
Flavor         0
Description    0
dtype: int64

In [12]:
x['Effects'].nunique()

1612

In [14]:
x['Effects'] = x['Effects'].str.split(',')

In [15]:
x['Effects']

0       [Creative, Energetic, Tingly, Euphoric, Relaxed]
1         [Relaxed, Aroused, Creative, Happy, Energetic]
2        [Uplifted, Happy, Relaxed, Energetic, Creative]
3          [Tingly, Creative, Hungry, Relaxed, Uplifted]
4        [Happy, Relaxed, Euphoric, Uplifted, Talkative]
                              ...                       
2346     [Happy, Uplifted, Relaxed, Euphoric, Energetic]
2347        [Relaxed, Happy, Euphoric, Uplifted, Sleepy]
2348       [Relaxed, Sleepy, Talkative, Euphoric, Happy]
2349          [Relaxed, Sleepy, Euphoric, Happy, Hungry]
2350          [Hungry, Relaxed, Uplifted, Happy, Sleepy]
Name: Effects, Length: 2277, dtype: object

In [21]:
x['Effects'] = x['Effects'].apply(lambda k: np.NaN if k[0] == 'None' else k)

In [25]:
x = x.dropna()

In [26]:
effects_set = set()
for lst in x['Effects']:
    for itm in lst:
        effects_set.add(itm)
effects_set

{'Aroused',
 'Creative',
 'Dry',
 'Energetic',
 'Euphoric',
 'Focused',
 'Giggly',
 'Happy',
 'Hungry',
 'Mouth',
 'Relaxed',
 'Sleepy',
 'Talkative',
 'Tingly',
 'Uplifted'}

In [28]:
x.isnull().sum()

Type           0
Rating         0
Effects        0
Flavor         0
Description    0
dtype: int64

In [36]:
temp = pd.get_dummies(x['Effects'].explode()).sum(level=0)

In [37]:
temp

,Aroused,Creative,Dry,Energetic,Euphoric,Focused,Giggly,Happy,Hungry,Mouth,Relaxed,Sleepy,Talkative,Tingly,Uplifted
0,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0
1,1,1,0,1,0,0,0,1,0,0,1,0,0,0,0
2,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1
3,0,1,0,0,0,0,0,0,1,0,1,0,0,1,1
4,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1
2347,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1
2348,0,0,0,0,1,0,0,1,0,0,1,1,1,0,0
2349,0,0,0,0,1,0,0,1,1,0,1,1,0,0,0


In [40]:
x = pd.concat([x, temp], axis=1)

In [42]:
x

,Type,Rating,Flavor,Description,Aroused,Creative,Dry,Energetic,Euphoric,Focused,Giggly,Happy,Hungry,Mouth,Relaxed,Sleepy,Talkative,Tingly,Uplifted
0,hybrid,4.0,"Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0
1,hybrid,4.7,"Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,1,1,0,1,0,0,0,1,0,0,1,0,0,0,0
2,sativa,4.4,"Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1
3,hybrid,4.2,"Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,0,1,0,0,0,0,0,0,1,0,1,0,0,1,1
4,hybrid,4.6,"Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...",0,0,0,0,1,0,0,1,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,hybrid,4.7,"Earthy,Woody,Pine",Zeus OG is a hybrid cross between Pineapple OG...,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1
2347,indica,4.6,"Sweet,Berry,Grape",Zkittlez is an indica-dominant mix of Grape Ap...,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1
2348,indica,5.0,"Earthy,Sweet,Spicy/Herbal",Zombie Kush by Ripper Seeds comes from two dif...,0,0,0,0,1,0,0,1,0,0,1,1,1,0,0
2349,indica,4.4,"Sweet,Earthy,Pungent",If you’re looking to transform into a flesh-ea...,0,0,0,0,1,0,0,1,1,0,1,1,0,0,0


In [43]:
x['Dry Mouth'] = x['Dry']

In [45]:
x = x.drop(columns=['Dry', 'Mouth'])
x

,Type,Rating,Flavor,Description,Aroused,Creative,Energetic,Euphoric,Focused,Giggly,Happy,Hungry,Relaxed,Sleepy,Talkative,Tingly,Uplifted,Dry Mouth
0,hybrid,4.0,"Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,0,1,1,1,0,0,0,0,1,0,0,1,0,0
1,hybrid,4.7,"Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,1,1,1,0,0,0,1,0,1,0,0,0,0,0
2,sativa,4.4,"Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,0,1,1,0,0,0,1,0,1,0,0,0,1,0
3,hybrid,4.2,"Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,0,1,0,0,0,0,0,1,1,0,0,1,1,0
4,hybrid,4.6,"Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...",0,0,0,1,0,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,hybrid,4.7,"Earthy,Woody,Pine",Zeus OG is a hybrid cross between Pineapple OG...,0,0,1,1,0,0,1,0,1,0,0,0,1,0
2347,indica,4.6,"Sweet,Berry,Grape",Zkittlez is an indica-dominant mix of Grape Ap...,0,0,0,1,0,0,1,0,1,1,0,0,1,0
2348,indica,5.0,"Earthy,Sweet,Spicy/Herbal",Zombie Kush by Ripper Seeds comes from two dif...,0,0,0,1,0,0,1,0,1,1,1,0,0,0
2349,indica,4.4,"Sweet,Earthy,Pungent",If you’re looking to transform into a flesh-ea...,0,0,0,1,0,0,1,1,1,1,0,0,0,0
